O'REILLY 에서 출판한 "트랜스포머를 활용한 자연어 처리" 책을 리뷰합니다.

### 감성 분석(Sentiment Analysis)

In [1]:
'''
자연어 처리(Natural Language Processing) 분야는 언어를 다루는 분야를 의미합니다. 
언어들 중에서는 영어, 불어, 한국어 등 다양한 언어가 있을 수 있고, 그 언어 내에서도 법률, 제품 설명, 레스토랑 리뷰 등 다양한 텍스트 종류가 존재합니다. 
레스토랑에 작성된 리뷰 데이터를 다루는 경우 사용자가 작성한 리뷰가 긍정적인 리뷰인지, 부정적인 리뷰인지 확인하고 싶을 것입니다. 

이와 같은 작업을 감성 분석(Sentiment Analysis)이라 부르며, 자연어 처리 분야 중 가장 대표적인 작업이라고 할 수 있습니다.

이번 챕터에서는 트랜스포머(Transformers)를 활용하여 감성 분석을 수행하는 방법에 대해서 알아볼 것입니다.
'''

text = """
    Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany.
    Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead!
    As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, 
    I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase.
    I expect to hear from you soon. Sincerely, Bumblebee.
"""

In [ ]:
# conda install -c conda-forge transformers==4.24.0
!pip install transformers==4.24.0

In [2]:
from transformers import pipeline

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
'''
pipeline() 함수를 호출하여 분석하고자 하는 작업 이름을 전달해 파이프라인 객체를 만들어 줍니다.
pipeline() 함수의 작업에는 아래와 같은 작업이 존재합니다.

task:
    audio-classification
    automatic-speech-recognition
    feature-extraction
    conversational
    fill-mask
    image-classification
    question-answering
    table-question-answering
    text2text-generation
    text-classification
    text-generation
    token-classification
    translation
    summarization
    zero-shot-classification

감성 분석 기법은 긍정 혹은 부정으로 분류하는 모델이 일반적이지만, pipeline(text-classification) 에서는 이진 분류(Binary Classification) 외에도
다중 분류(Multi-class Classification) 모델도 지원합니다.
'''

classifier = pipeline('text-classification')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading: 100%|██████████| 629/629 [00:00<00:00, 645kB/s]
Downloading: 100%|██████████| 268M/268M [00:22<00:00, 11.9MB/s] 
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 42.4kB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 303kB/s]  


In [7]:
import pandas as pd 

In [8]:
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


### 개체명 인식(Named Entity Recognition, NER)

In [9]:
'''
자연어 처리에서 제품, 장소, 사람 등과 같은 실제 객체를 개체명(Named Entity)이라고 하며, 
이런 개체명을 텍스트에서 추출하는 작업을 개체명 인식(Named Entity Recognition, NER)이라고 한다. 
텍스트를 분석할 때 "서울대학교에서는 올해 100명의 신입생이 입학하였다." 라는 글을 기계가 분석할 때 "서울", "대학교"로 분리해서 인식하게 된다면, 
서울에 있는 전체 대학의 신입생이 100명으로 잘못 인식될 수 있다. 따라서, "서울대학교"라는 개체명을 제대로 인식하는 작업이 필요하다. 

ORG: 조직
LOC: 위치
PER: 사람
MISC: 그외
'''

ner_tagger = pipeline('ner', aggregation_strategy='simple')
outputs = ner_tagger(text)
pd.DataFrame(outputs) 

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading: 100%|██████████| 998/998 [00:00<00:00, 1.10MB/s]
Downloading: 100%|██████████| 1.33G/1.33G [01:52<00:00, 11.8MB/s]
Downloading: 100%|██████████| 60.0/60.0 [00:00<00:00, 64.9kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 573kB/s] 


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,10,16
1,MISC,0.990859,Optimus Prime,41,54
2,LOC,0.999755,Germany,95,102
3,MISC,0.556570,Mega,217,221
4,PER,0.590256,##tron,221,225
5,ORG,0.669693,Decept,266,272
6,MISC,0.498349,##icons,272,277
7,MISC,0.775362,Megatron,368,376
8,MISC,0.987854,Optimus Prime,385,398
9,PER,0.812096,Bumblebee,524,533


### 질문 답변(Question and Answering, Q&A)

In [10]:
'''
Q&A는 질문이 들어왔을 때 본문(텍스트)을 참고하여 답변을 하는 작업이다.
질문 답변에는 추출 답변과 추상 답변이 있다. 
추출 답변은 본문에 있는 답변에서 답변이라고 생각되는 문장 구절을 추출하는 방법이다.
반면에, 추상 답변은 추상적으로, 즉 사람과 유사한 방식으로 문맥을 이해하여 답변을 하는 방식이다.
'''

reader = pipeline('question-answering')
question = 'What does the customer want?'
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading: 100%|██████████| 473/473 [00:00<00:00, 691kB/s]
Downloading: 100%|██████████| 261M/261M [00:21<00:00, 11.9MB/s] 
Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 33.3kB/s]
Downloading: 100%|██████████| 213k/213k [00:00<00:00, 368kB/s]  
Downloading: 100%|██████████| 436k/436k [00:01<00:00, 259kB/s] 


,score,start,end,answer
0,0.631292,353,376,an exchange of Megatron


### 텍스트 요약(Text Summarization)

In [13]:
'''
텍스트 요약은 질문 답변과 유사하게 추상 요약과 추출 요약이 있다.
추상 요약은 사람과 비슷하게 요약하는 것을 의미하며, 추출 요약은 본문에 있는 내용을 순서에 맞게 짜깁기해서 요약하는 방식이다.
추출 요약보다 추상 요약이 훨씬 어렵다. 
'''


summarizer = pipeline('summarization')
outputs = summarizer(text, max_length=56, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee demands an exchange of Megatron for an Optimus Prime figure he bought from Amazon. The Decepticons are a lifelong enemy of the Decepticon, and he wants the figure exchanged for Megatron. The Transformers figure was sent to him from an online


### 번역(Translation)

In [6]:
'''
기계 번역은 Sequence-to-Sequence 모델 중 가장 대표적인 작업이다. 
pipeline("translation_xx_to_yy", model="any")를 이용해 언어를 원하는 언어로 번역할 수 있다.
xx에는 텍스트의 언어에 대한 약자를 입력하고, yy에는 번역하고 싶은 언어에 대한 약자를 입력하면 된다.
'''
import sentencepiece
translator = pipeline('translation_en_to_de', model='Helsinki-NLP/opus-mt-en-de')
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

Downloading: 100%|██████████| 768k/768k [00:01<00:00, 397kB/s] 
Downloading: 100%|██████████| 797k/797k [00:02<00:00, 356kB/s]  
Downloading: 100%|██████████| 1.27M/1.27M [00:02<00:00, 451kB/s]


Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur habe ich bestellt. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, bald von Ihnen zu hören. Aufrichtig, Bumblebee.


### 텍스트 생성(Text Generation)

In [9]:
'''
텍스트 생성은 위 질문을 통해 아래에 자동적으로 답변을 생성할 수 있는 작업입니다. 
아래의 간단한 코드로 위 질문에 대한 답변을 쉽게 생성할 수 있습니다.
'''

generator = pipeline('text-generation')
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + '\n\nCustomer service response:\n' + response 
outputs = generator(prompt, max_length=200)

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
print(outputs[0]['generated_text'])


    Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany.
    Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead!
    As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, 
    I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase.
    I expect to hear from you soon. Sincerely, Bumblebee.


Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I was looking for a collector's item that is very similar to the Autobots. The picture here is taken directly from the package. To my disappointment, the pictures shown here were sent from you after you responded. I am sorry
